In [ ]:
import pandas as pd
import numpy as np
import os
import time
import datetime
from openai import OpenAI
import pymssql
import re
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from threading import Thread
import functools
from tableone import TableOne
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu as mwu
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.statistics import multivariate_logrank_test
from sksurv.nonparametric import kaplan_meier_estimator
from sklearn.metrics import roc_curve, auc
import random
from threading import Thread
import functools
import google.generativeai as genai

conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
genai.configure(api_key="") # enter google cloud api key

In [ ]:
generation_config = {
  "temperature": 0,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

In [ ]:
rand_df_temp=pd.read_csv('20231227_synthetic_20000_main_experiments.csv')
rand_df_temp.drop(['input_text_0','input_text_2','input_text_3','input_text_4','input_text_5','input_text_6','input_text_7','input_text_8','input_text_9','input_text_10'],axis=1,inplace=True)

rand_df_temp=rand_df_temp.iloc[:20000]
rand_df=rand_df_temp.copy()
for i in range(9):
    rand_df=pd.concat([rand_df,rand_df_temp])
rand_df.reset_index(inplace=True,drop=True)
rand_df['eid']=range(200000)

In [ ]:
usercontent_global1='''Estimate the risk (in percentages) of developing a cardiovascular disease within 10 years for the person below.

'''
usercontent_global2='''

Please answer exactly in the format below, without blank lines, and no further information or answer is required.
Risk percentage=(in percentages, round to one decimal place)'''

input_text=rand_df['input_text_1']
eid=rand_df['eid']

def timeout(timeout):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            res = [Exception('function [%s] timeout [%s seconds] exceeded!' % (func.__name__, timeout))]
            def newFunc():
                try:
                    res[0] = func(*args, **kwargs)
                except Exception as e:
                    res[0] = e
            t = Thread(target=newFunc)
            t.daemon = True
            try:
                t.start()
                t.join(timeout)
            except Exception as je:
                print ('error starting thread')
                raise je
            ret = res[0]
            #if isinstance(ret, BaseException):
            #    raise ret
            return ret
        return wrapper
    return deco

@timeout(0.1)
def ChatGPT_main(i,temper,responsenum):
    if input_text[i]!='':
        try:
            usercontent=usercontent_global1
            usercontent+=input_text[i]
            usercontent+=usercontent_global2

            model = genai.GenerativeModel(
              model_name="gemini-1.5-pro-002",
              generation_config=generation_config,
            )

            chat_session = model.start_chat(
              history=[
              ]
            )
            
            response = chat_session.send_message(usercontent)
            responsetext=response.text
            for j in range(1):

                temp1='blank'
                temp2=responsetext.split('=')[1]
                temp3='blank'
                temp4='blank'
                temp5='blank'
                
                conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
                with conn:
                    with conn.cursor() as cur:
                        cur.execute(sql, (str(eid[i]),temp1,temp2,temp3,temp4,temp5))
                        conn.commit()

        except:
            temp1=np.nan
            temp2=np.nan
            temp3=np.nan
            temp4=np.nan
            temp5=np.nan
            
    else:
        temp1=np.nan
        temp2=np.nan
        temp3=np.nan
        temp4=np.nan
        temp5=np.nan
        


In [ ]:
responsenumnumnum=1
for experimentnum in range(1,2):
    print('=========================================================')
    print('experimentnum',experimentnum)
    input_text=rand_df['input_text_1']
    ############################
    table_name="20241029_GEMINI_CVD1"
    iterations=1
    upto=200000
    ############################
    for temper in [0.0]:
        time1=datetime.datetime.now()
        while True:
            toggle=0
            for i in range(iterations):   
                newtablaname=table_name + '_' + str(int(experimentnum)) + '_' + str(i)
                try:
                    sql_createtable="CREATE TABLE [" + newtablaname +"""] 
                    (
                        eid    NVARCHAR(20),
                        system    NVARCHAR(max),
                        score      NVARCHAR(max),
                        category     NVARCHAR(max) ,
                        framingham  NVARCHAR(20),
                        ACC_AHA  NVARCHAR(20)
                    )

                    """
                    conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
                    with conn:
                        with conn.cursor() as cur:
                            cur.execute(sql_createtable)
                            conn.commit()
                    time.sleep(1)  
                except:
                    pass

                sql_statement="select * from ["+ newtablaname + "]"
                conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
                datasql = pd.read_sql(sql=sql_statement, con=conn)
                datasql=datasql.astype({'eid':int})
                cnt_df=datasql.groupby('eid')['score'].count()
                tempcount=0
                for j in range(upto):
                    try:
                        cnt=cnt_df[j]
                        if cnt<responsenumnumnum:
                            addcnt=int(responsenumnumnum-cnt)
                            sql = "INSERT INTO [" + newtablaname+"] (eid, system,score,category,framingham,ACC_AHA) VALUES (%s, %s, %s, %s, %s, %s)"
                            ChatGPT_main(j,temper,addcnt)
                            tempcount+=1

                    except:
                        sql = "INSERT INTO [" + newtablaname+"] (eid, system,score,category,framingham,ACC_AHA) VALUES (%s, %s, %s, %s, %s, %s)"
                        ChatGPT_main(j,temper,responsenumnumnum)
                        tempcount+=1
                    if j%100==0:
                        print(j)

                if tempcount==0:
                    toggle=1
            if toggle==1:
                break
            time.sleep(20)

        time.sleep(20)


        time2=datetime.datetime.now()
        print(time2-time1)
        #print('except_else_num = ',except_else_num)

In [ ]:
table_name="20241029_GEMINI_CVD1"

In [ ]:
from sklearn.linear_model import LinearRegression
from lifelines.utils import concordance_index
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

gpt_cindex=[]
validnum=[]
for experimentnum in range(1,2):
    print('=========================================================')
    print('experimentnum',experimentnum)
    rand_df=pd.read_csv('20231227_synthetic_20000_main_experiments.csv')
    input_text=rand_df['input_text_'+str(experimentnum)]
    ############################

    iterations=1
    upto=20000
    ############################
    resultsdf=pd.DataFrame(columns=['validnum','age', 'HDL', 'LDL', 'TG', 'HbA1c', 'Cr', 'Urate', 'CRP', 'SBP', 'DBP', 'BMI', 'WHratio', 'sex', 'HTN', 'DM', 'Dyslipidemia', 'Afib', 'CKD', 'CVDfamily', 'smoking','avg_gpt_score','c-index gpt','c-index acc/aha','c-index framingham'])


    for temper in [0.0]:
        templist=[]
        print('=========temperature========',temper)
        for i in range(iterations):
            newtablename=table_name + '_' + str(int(experimentnum)) + '_' + str(i)

            sql_statement="select * from ["+ newtablename + "]"
            conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
            globals()['data{}'.format(i)] = pd.read_sql(sql=sql_statement, con=conn)


        for j in range(iterations):
            dat=globals()['data{}'.format(j)]

            eid=dat['eid']
            score_gpt=dat['score']
            category=dat['category']
            score_framingham=dat['framingham']
            score_acc_aha=dat['ACC_AHA']

            score_gpt2=[]

            for i in range(len(score_gpt)):
                try:
                    score_gpt2.append(float(re.findall("\d+[.]\d+[%]",score_gpt[i])[0].split('%')[0]))
                except:
                    try:
                        score_gpt2.append(float(re.findall("\d+[%]",score_gpt[i])[0].split('%')[0]))
                    except:
                        try:
                            score_gpt2.append(float(re.findall("\d+[.]\d+",score_gpt[i])[0]))
                        except:
                            try:
                                score_gpt2.append(float(re.findall("\d+",score_gpt[i])[-1]))
                            except:
                                score_gpt2.append(np.nan)
                                #score_gpt2.append(float(score_framingham[i]))
                                #print(score_gpt[i],'====')


        data_final=pd.DataFrame({'eid':data0['eid'],'score':score_gpt2})
        data_final['eid']=data_final['eid'].astype(int)
        print(len(data_final))
        data_final=data_final.dropna()
        data_final.reset_index(inplace=True,drop=True)
        
        
        rand_df_temp=pd.read_csv('20231227_synthetic_20000_main_experiments.csv')
        rand_df_temp.drop(['input_text_0','input_text_2','input_text_3','input_text_4','input_text_5','input_text_6','input_text_7','input_text_8','input_text_9','input_text_10'],axis=1,inplace=True)

        rand_df_temp=rand_df_temp.iloc[:20000]
        rand_df=rand_df_temp.copy()
        for i in range(9):
            rand_df=pd.concat([rand_df,rand_df_temp])
        rand_df.reset_index(inplace=True,drop=True)
        rand_df['eid']=range(200000)
        

        #rand_df=pd.read_csv('20231226_synthetic_20000.csv')
        #rand_df=rand_df.iloc[:upto]
        
        totalchol=[]
        for i in range(len(rand_df)):
            tcholtemp=rand_df['HDL'][i]+rand_df['LDL'][i]+rand_df['TG'][i]/5
            totalchol.append(round(tcholtemp,1))
        rand_df['totalchol']=totalchol

        #data_final_groupmean=data_final.groupby(['eid'],as_index=False).mean()

        df_merged=pd.merge(data_final,rand_df,how='inner',on='eid')
        print(len(df_merged))
        validnum.append(len(df_merged))
        templist.append(len(df_merged))
        columnslisttemp=list(df_merged.columns)
        columnslisttemp=columnslisttemp[2:]
        columnslisttemp=columnslisttemp[:-1]
        #print(columnslisttemp)
        #X=df_merged[['age','sex','HDL','LDL','TG','Chol','SBP','DBP','BMI','HTN','DM','smoking']]
        #X=df_merged[columnslisttemp]
        #y=df_merged['score']

        #line_fitter = LinearRegression()
        #line_fitter.fit(X, y)

        #print(line_fitter.coef_)
        #templist+=list(line_fitter.coef_)

        df_final=pd.read_csv('20231122_df_final.csv')

        columnslist=[]
        columnslist_cleaned=[]
        if experimentnum in [0,1,2,3,4,5,6,7]:
            columnslist.append('age')
            columnslist.append('sex')
            columnslist.append('total_chol')
            columnslist.append('HDL')
            columnslist_cleaned.append('age')
            columnslist_cleaned.append('sex')
            columnslist_cleaned.append('totalchol')
            columnslist_cleaned.append('HDL')
        if experimentnum in [1,2,3,5]:
            columnslist.append('LDL')
            columnslist.append('TG')
            columnslist_cleaned.append('LDL')
            columnslist_cleaned.append('TG')
        if experimentnum in [0,1,2,3,4,5,6,7]:
            columnslist.append('SBP')
            columnslist_cleaned.append('SBP')
        if experimentnum in [1,2,4,6]:
            columnslist.append('DBP')
            columnslist_cleaned.append('DBP')
        if experimentnum in [0,1,2,3,4,5,6,7]:
            columnslist.append('med_final')
            columnslist.append('DM_final')
            columnslist_cleaned.append('HTN')
            columnslist_cleaned.append('DM')
        if experimentnum in [1,2,4,6]:
            columnslist.append('BMI_x')
            columnslist_cleaned.append('BMI')
        if experimentnum in [0,1,2,3,4,5,6,7]:
            columnslist.append('cig_final')
            columnslist_cleaned.append('smoking')
        if experimentnum in [1,3,4,7]:
            columnslist.append('E78_dyslipidemia_binary')
            columnslist.append('I48_afib_binary')
            columnslist.append('N18_CKD_binary')
            columnslist_cleaned.append('Dyslipidemia')
            columnslist_cleaned.append('Afib')
            columnslist_cleaned.append('CKD')
        if experimentnum in [1,2,3,5]:
            columnslist.append('HbA1c')
            columnslist.append('Creatinine')
            columnslist.append('urate')
            columnslist.append('C_reactive_ptn')
            columnslist_cleaned.append('HbA1c')
            columnslist_cleaned.append('Cr')
            columnslist_cleaned.append('Urate')
            columnslist_cleaned.append('CRP')
        if experimentnum in [1,3,4,7]:
            columnslist.append('CVD_firstdegree')
            columnslist_cleaned.append('CVDfamily')
        columnslist.append('firstoccur_event')
        if experimentnum in [1,2,4,6]:
            columnslist.append('WHratio')
            columnslist_cleaned.append('WHratio')
        columnslist.append('eid')
        columnslist.append('Framingham')
        columnslist.append('ACC/AHA')
        columnslist.append('datediff_firstoccur_int2')
        df_final2=df_final[columnslist]
        df_final2.dropna(inplace=True)
        df_final2.reset_index(inplace=True,drop=True)
        firstoccur_event=[]
        for i in range(len(df_final2)):
            if df_final2['firstoccur_event'][i]==True:
                firstoccur_event.append(1)
            else:
                firstoccur_event.append(0)
        df_final2['firstoccur_event']=firstoccur_event

        #df2=df_final2[['eid','firstoccur_event','datediff_firstoccur_int2','ACC/AHA','Framingham','age', 'HDL', 'LDL', 'TG', 'HbA1c', 'Creatinine', 'urate', 'C_reactive_ptn', 'SBP', 'DBP', 'BMI_x', 'WHratio', 'sex', 'med_final', 'DM_final', 'E78_dyslipidemia_binary', 'I48_afib_binary', 'N18_CKD_binary', 'CVD_firstdegree', 'cig_final']]
        df2=df_final2
        df2=df2.dropna()
        print(len(df2))
        
        columnslist2=columnslist.copy()
        columnslist2.remove('firstoccur_event')
        columnslist2.remove('eid')
        columnslist2.remove('Framingham')
        columnslist2.remove('ACC/AHA')
        columnslist2.remove('datediff_firstoccur_int2')
        
        
        
        X=df_merged[columnslist_cleaned]
        X.columns=columnslist2
        y=df_merged['score']
        
        cig_exsmoker=[]
        cig_currentsmoker=[]
        for aa in range(len(X)):
            if X['cig_final'][aa]==0:
                cig_exsmoker.append(0)
                cig_currentsmoker.append(0)
            elif X['cig_final'][aa]==1:
                cig_exsmoker.append(1)
                cig_currentsmoker.append(0)
            else:
                cig_exsmoker.append(0)
                cig_currentsmoker.append(1)
        X['cig_exsmoker']=cig_exsmoker
        X['cig_currentsmoker']=cig_currentsmoker
        X.drop(['cig_final'],axis=1,inplace=True)
        columnslist3=columnslist2.copy()
        columnslist2.remove('cig_final')
        columnslist2.append('cig_exsmoker')
        columnslist2.append('cig_currentsmoker')
        if experimentnum in [1,2,3,5]:
            columnslist2.remove('total_chol')
            X.drop(['total_chol'],axis=1,inplace=True)
        #print(X.columns)
        print(columnslist2)
        line_fitter = LinearRegression()
        line_fitter.fit(X, y)

        print(line_fitter.coef_)
        templist+=list(line_fitter.coef_)
        X_temp=X.copy()
        X_temp['y']=y
        X_temp.to_csv('20241029_GEMINI_CVD_R.csv',index=False)
   
        
        #X=df2[['age', 'HDL', 'LDL', 'TG', 'HbA1c', 'Creatinine', 'urate','C_reactive_ptn', 'SBP', 'DBP', 'BMI_x', 'WHratio', 'sex', 'med_final', 'DM_final', 'E78_dyslipidemia_binary', 'I48_afib_binary', 'N18_CKD_binary', 'CVD_firstdegree', 'cig_final']]
        #X.columns=['age', 'HDL', 'LDL', 'TG', 'HbA1c', 'Cr', 'Urate', 'CRP', 'SBP', 'DBP', 'BMI', 'WHratio', 'sex', 'HTN', 'DM', 'Dyslipidemia', 'Afib', 'CKD', 'CVDfamily', 'smoking']
        if experimentnum in [1,2,3,5]:
            columnslist3.remove('total_chol')
        X=df2[columnslist3]
        
        cig_exsmoker=[]
        cig_currentsmoker=[]
        for aa in range(len(X)):
            if X['cig_final'][aa]==0:
                cig_exsmoker.append(0)
                cig_currentsmoker.append(0)
            elif X['cig_final'][aa]==1:
                cig_exsmoker.append(1)
                cig_currentsmoker.append(0)
            else:
                cig_exsmoker.append(0)
                cig_currentsmoker.append(1)
        X['cig_exsmoker']=cig_exsmoker
        X['cig_currentsmoker']=cig_currentsmoker
        X.drop(['cig_final'],axis=1,inplace=True)

        
        y=line_fitter.predict(X)
        df2['score']=y
  
        
        df_final=df2

        templist.append(np.mean(df_final['score']))

        fpr,tpr,thres=roc_curve(df_final['firstoccur_event'],df_final['score'])
        print('event percentage',np.mean(df_final['firstoccur_event']))
        print('average gpt score',np.mean(df_final['score']))
        print('average accaha',np.mean(df_final['ACC/AHA']))
        print('average framingham',np.mean(df_final['Framingham']))
        print('gpt auroc',auc(fpr, tpr))
        fpr,tpr,thres=roc_curve(df_final['firstoccur_event'],df_final['ACC/AHA'])
        print('accaha auroc',auc(fpr, tpr))
        fpr,tpr,thres=roc_curve(df_final['firstoccur_event'],df_final['Framingham'])
        print('framingham auroc',auc(fpr, tpr))

        templist.append(concordance_index(df_final['datediff_firstoccur_int2'], -df_final['score'], df_final['firstoccur_event']))
        templist.append(concordance_index(df_final['datediff_firstoccur_int2'], -df_final['ACC/AHA'], df_final['firstoccur_event']))
        templist.append(concordance_index(df_final['datediff_firstoccur_int2'], -df_final['Framingham'], df_final['firstoccur_event']))
        
        #resultsdf.loc[len(resultsdf)]=templist
        gpt_cindex.append(concordance_index(df_final['datediff_firstoccur_int2'], -df_final['score'], df_final['firstoccur_event']))
        print('c-index gpt',concordance_index(df_final['datediff_firstoccur_int2'], -df_final['score'], df_final['firstoccur_event']))
        print('c-index acc/aha',concordance_index(df_final['datediff_firstoccur_int2'], -df_final['ACC/AHA'], df_final['firstoccur_event']))
        print('c-index framingham',concordance_index(df_final['datediff_firstoccur_int2'], -df_final['Framingham'], df_final['firstoccur_event']))
        df_final.to_csv('20241029_GEMINI_CVD.csv',index=False)

In [ ]:
CINDEX_cols=['Cindex_LLM','Cindex_PCE','Cindex_FRS']
rows_list=[]

for iiii in range(2000):
    df_final_resample=df_final.sample(frac=1, replace=True)


    cindex_LLM=concordance_index(df_final_resample['datediff_firstoccur_int2'], -df_final_resample['score'], df_final_resample['firstoccur_event'])
    cindex_PCE=concordance_index(df_final_resample['datediff_firstoccur_int2'], -df_final_resample['ACC/AHA'], df_final_resample['firstoccur_event'])
    cindex_FRS=concordance_index(df_final_resample['datediff_firstoccur_int2'], -df_final_resample['Framingham'], df_final_resample['firstoccur_event'])
    
    row = [
        cindex_LLM,
        cindex_PCE,
        cindex_FRS
    ]
    rows_list.append(row)
    if iiii%100==0:
        print(iiii)
CINDEX_df = pd.DataFrame(rows_list, columns=CINDEX_cols)    
CINDEX_df.to_csv('20241030_GEMINI_CVD_CINDEX_bootstrap.csv',index=False)

In [ ]:
CINDEX_df['diff_PCE']=CINDEX_df['Cindex_LLM']-CINDEX_df['Cindex_PCE']
CINDEX_df['diff_FRS']=CINDEX_df['Cindex_LLM']-CINDEX_df['Cindex_FRS']

In [ ]:
print(np.percentile(CINDEX_df['diff_PCE'],2.5))
print(np.percentile(CINDEX_df['diff_PCE'],97.5))

In [ ]:
print(np.percentile(CINDEX_df['diff_FRS'],2.5))
print(np.percentile(CINDEX_df['diff_FRS'],97.5))